# AI Usage & Performance Analysis – Esfera System

End-to-end analysis of usage, quality, and operational metrics of an internal AI assistant.

## 1. Business Context

Esfera is a construction management software that includes a conversational AI assistant.

Users interact with the AI to:
- Retrieve project information
- Interpret operational data
- Generate summaries

At this stage, the platform has a small number of active clients, and part of the usage data corresponds to internal testing and training sessions.

Given this early context, the goal of this analysis is not to measure business impact at scale, but to **understand initial usage behavior, response quality, system stability, and cost dynamics** of the AI feature.

This exploration aims to provide a structured baseline that can **support future monitoring and decision-making as adoption grows.**

Esfera operates in a **Spanish-speaking market**, and all user interactions analyzed in this project are in Spanish.

## 2. Objectives

The objectives of this analysis are:

1. Understand how users interact with the AI within the platform
2.	Evaluate how effectively the AI resolves user queries
3.	Identify friction patterns vs. legitimate exploratory usage
4.	Define clear and defensible KPIs for AI performance
5.	Build a structured dataset ready for dashboarding and ongoing monitoring

## 3. Data Sources

This analysis is based on two main data views extracted from the Esfera platform.

### 3.1 AI Interaction Logs

The primary dataset contains conversational interaction logs between users and the AI assistant.  
Main fields used:

- `UserId`
- `idEmpresa`
- `event_ts`
- `message`
- `response`
- `Error`

This dataset captures when users interact with the AI, what they ask, and whether the system returns an explicit technical error.


### 3.2 AI Performance & Token Metrics

A second data view was incorporated to analyze technical performance and operational cost.  
Key fields include:

- `duration_ms` (total response time in milliseconds)
- `ttft` (time to first token, in milliseconds)
- `prompt_token_count`
- `candidates_token_count`
- Derived cost per query (based on token usage)


### Data Considerations & Limitations

- There is no structured user satisfaction field.
- Query types are not pre-classified.
- Early usage includes internal testing and training activity.
- Some KPIs (e.g., satisfaction proxy, cost per query) were engineered during the analysis process rather than directly extracted.

All performance and behavioral metrics presented in this notebook are derived from these structured logs.

## 4. Methodological Decisions

Before calculating KPIs, several structural decisions were made to ensure consistent and defensible metrics.

### Session Definition

A session is defined as consecutive interactions from the same user separated by less than 10 minutes.  
If the time gap exceeds 10 minutes, a new session begins.

This allows analysis at session-level instead of isolated messages.

### Real Query Definition

Not all messages represent meaningful analytical intent.

A real query is defined as a message that:
- Has 20 or more characters, or
- Contains a question mark ("?")

This filtering removes greetings, trivial messages, and noise without excluding short but intentional queries.

### Proxy-Based Satisfaction

Since there is no explicit user satisfaction field, satisfaction is approximated using behavioral signals.

A session is considered successfully resolved when:
- It contains 1–2 real queries, and
- No explicit system error is present.

### Performance & Cost Metrics

Unlike the initial dataset exploration phase, the final analysis includes:

- Response time metrics (average duration in seconds)
- Time to first token (TTFT)
- Estimated cost per query based on token usage

These metrics were derived from the enriched logs view containing token and timing information.

## 5. Exploratory Analysis: Types of AI Queries

The goal of this section is to understand what users primarily use the AI for inside the Esfera system.

Rather than focusing on performance metrics, this analysis provides **contextual insight into user intent and usage patterns.**


### 5.1 Query Classification Approach

Since there is no structured field indicating the type of query, a keyword-based classification was implemented.

Each message was evaluated using heuristic rules based on recurring terms such as:

- **Financial terms** (cost, budget, supplier, inventory)
- **Project progress** terms (progress, advancement, status)
- **Reporting terms** (summary, report, chart/graph)
- **System guidance questions** (how to use, modules, features/functions)
- **Planning-related terms** (schedule, deadline, date)
- **Operational actions** (enter, edit, modify)
- **Support requests** (support, agent/representative)

Each message can activate one or more thematic flags depending on detected keywords.

This classification is exploratory and descriptive, not a production-level NLP model.

***Note:*** The original user messages are written in Spanish, as **Esfera operates in a Spanish-speaking business** environment. 

Therefore, keyword-based classification rules are also defined in Spanish to ensure semantic consistency with the source data.

### 5.2 SQL Used to Calculate Distribution

The following BigQuery SQL was used to classify queries and compute their distribution by topic.

Note: This query was **executed in Google BigQuery**. It is shown here for documentation purposes.

```sql
WITH base AS (
  SELECT
    IF(LOWER(message) LIKE '%en que proyecto%' OR LOWER(message) LIKE '%que proyecto%' OR
       LOWER(message) LIKE '%modulos%' OR LOWER(message) LIKE '%funciones%' OR
       LOWER(message) LIKE '%que puedo consultar%' OR LOWER(message) LIKE '%como funciona%' OR
       LOWER(message) LIKE '%como usar%', 1, 0) AS dim_orientacion,

    IF(LOWER(message) LIKE '%avance%' OR LOWER(message) LIKE '%progreso%' OR
       LOWER(message) LIKE '%como vamos%' OR LOWER(message) LIKE '%estado%', 1, 0) AS dim_avance,

    IF(LOWER(message) LIKE '%presupuesto%' OR LOWER(message) LIKE '%costo%' OR
       LOWER(message) LIKE '%gasto%' OR LOWER(message) LIKE '%material%' OR
       LOWER(message) LIKE '%proveedor%' OR LOWER(message) LIKE '%pago%' OR
       LOWER(message) LIKE '%stock%' OR LOWER(message) LIKE '%inventario%' OR
       LOWER(message) LIKE '%item mas caro%' OR LOWER(message) LIKE '%item mas barato%', 1, 0) AS dim_finanzas,

    IF(LOWER(message) LIKE '%cronograma%' OR LOWER(message) LIKE '%plazo%' OR
       LOWER(message) LIKE '%fecha%' OR LOWER(message) LIKE '%cuando%', 1, 0) AS dim_planificacion,

    IF(LOWER(message) LIKE '%ingresar%' OR LOWER(message) LIKE '%cargar%' OR
       LOWER(message) LIKE '%editar%' OR LOWER(message) LIKE '%modificar%', 1, 0) AS dim_operacion,

    IF(LOWER(message) LIKE '%resum%' OR LOWER(message) LIKE '%reporte%' OR
       LOWER(message) LIKE '%grafico%' OR LOWER(message) LIKE '%dashboard%', 1, 0) AS dim_reportes,

    IF(LOWER(message) LIKE '%hablar con%' OR LOWER(message) LIKE '%soporte%' OR
       LOWER(message) LIKE '%agente%', 1, 0) AS dim_soporte

  FROM `conductive-bank-469015-r4.esfera_IA.logs_ai_esfera_producto_vw`
  WHERE message IS NOT NULL
),

totales AS (
  SELECT
    SUM(dim_orientacion) AS orientacion,
    SUM(dim_avance) AS avance,
    SUM(dim_finanzas) AS finanzas,
    SUM(dim_planificacion) AS planificacion,
    SUM(dim_operacion) AS operacion,
    SUM(dim_reportes) AS reportes,
    SUM(dim_soporte) AS soporte
  FROM base
),

total_general AS (
  SELECT
    orientacion + avance + finanzas + planificacion + operacion + reportes + soporte AS total
  FROM totales
)

SELECT
  'Orientación' AS dimension,
  ROUND(100 * orientacion / total, 1) AS pct_tema
FROM totales, total_general

UNION ALL
SELECT 'Avance', ROUND(100 * avance / total, 1) FROM totales, total_general
UNION ALL
SELECT 'Finanzas', ROUND(100 * finanzas / total, 1) FROM totales, total_general
UNION ALL
SELECT 'Planificación', ROUND(100 * planificacion / total, 1) FROM totales, total_general
UNION ALL
SELECT 'Operación', ROUND(100 * operacion / total, 1) FROM totales, total_general
UNION ALL
SELECT 'Reportes', ROUND(100 * reportes / total, 1) FROM totales, total_general
UNION ALL
SELECT 'Soporte', ROUND(100 * soporte / total, 1) FROM totales, total_general;
```

### 5.3 Distribution of Queries by Topic

After applying the classification logic, the distribution of queries by topic was calculated as follows:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Data
data = {
    "Query Type": [
        "Finance",
        "Project Progress",
        "Reporting",
        "System Guidance",
        "Planning",
        "Operations",
        "Support"
    ],
    "Percentage": [31.3, 30.8, 28.2, 6.2, 1.5, 1.5, 0.5]
}

df = pd.DataFrame(data)

# Sort from highest to lowest
df = df.sort_values("Percentage", ascending=True)

# Professional color palette (more neutral)
colors = [
    "#d62728",   # Support
    "#8c564b",   # Operations
    "#bcbd22",   # Planning
    "#7f7f7f",   # System Guidance
    "#17becf",   # Reporting
    "#2ca02c",   # Project Progress
    "#1f77b4"    # Finance
]

# Plot
plt.figure(figsize=(8, 5))
plt.barh(
    df["Query Type"],
    df["Percentage"],
    color=colors
)

plt.xlabel("Percentage of Queries")
plt.title("AI Query Distribution by Business Dimension")

# Percentage labels
for index, value in enumerate(df["Percentage"]):
    plt.text(value + 0.5, index, f"{value:.1f}%", va="center")

plt.tight_layout()
plt.show()


### 5.4 Key Insight

The AI is primarily used to **analyze and interpret financial and project management information**, rather than to execute operational tasks or resolve technical issues.

Financial, project progress, and reporting-related queries account for the vast majority of interactions. This suggests that **users perceive the AI as an analytical support tool** rather than an operational assistant.

Lower percentages in operational and support-related queries may reflect either limited feature awareness or a natural usage pattern centered on high-level decision-making.

### 5.5 Methodological Note

This classification is exploratory and heuristic-based, derived from keyword matching and manual review of representative samples.

Categories are not mutually exclusive at the message level; instead, they provide **directional insight into dominant themes across interactions**.

At this stage, the classification is implemented for analytical exploration. However, its structure is intentionally designed to be replicable and can later be automated using a dedicated NLP model within the live dashboard environment.

### 5.6 Notes on Potential Automation

The current topic classification is based on rule-based keyword matching, designed for exploratory and descriptive analysis.

While this approach is sufficient to understand high-level usage patterns, it has limitations:
- Queries may belong to more than one category
- Keyword rules require manual maintenance
- Contextual meaning is not fully captured

If this analysis were to evolve into a production monitoring system, a supervised text classification model could replace the heuristic rules to provide more consistent tagging.

For the scope of this project, the objective is to establish a defensible analytical framework and demonstrate how query behavior can be structured and interpreted.

## 6. Cross-Sectional Insights

Based on exploratory and behavioral analysis, the following high-level insights were identified:

1. The AI is primarily used as an **analytical assistant** rather than a search tool or operational interface.

2. A large portion of user queries are **complex or multi-dimensional**, often combining financial, operational, and progress-related aspects.

3. Friction appears to stem less from technical errors and more from **clarity, synthesis, and information prioritization.**

4. Users actively explore project data, but first responses do not always fully resolve their intent.

5. There is a clear opportunity to improve user experience through more structured summaries and clearer prioritization of key information.

## 7. Core KPIs

The following KPIs were defined to measure usage, quality, efficiency, and cost of the AI system.

### KPI 1 — Satisfactory Response Rate

**Value:** 39%

**Definition:**  
Percentage of sessions resolved within 1 or 2 real queries, without explicit technical errors.

**Interpretation:**  
Approximately 4 out of 10 conversations are resolved clearly and efficiently in the first interaction cycle.

#### SQL Used to Calculate KPI 1

The following query was executed in BigQuery to calculate the Satisfactory Response Rate:

```sql
    WITH base AS (
  SELECT
    UserId,
    event_ts,
    message,
    Error,
    LAG(event_ts) OVER (
      PARTITION BY UserId
      ORDER BY event_ts
    ) AS prev_ts
  FROM `conductive-bank-469015-r4.esfera_IA.logs_ai_esfera_producto_vw`
),

-- Identify new sessions based on 10-minute inactivity threshold
sessions AS (
  SELECT
    UserId,
    event_ts,
    message,
    Error,
    IF(
      prev_ts IS NULL
      OR TIMESTAMP_DIFF(event_ts, prev_ts, MINUTE) > 10,
      1,
      0
    ) AS new_session_flag
  FROM base
),

-- Assign session IDs per user
session_ids AS (
  SELECT
    UserId,
    event_ts,
    message,
    Error,
    SUM(new_session_flag) OVER (
      PARTITION BY UserId
      ORDER BY event_ts
    ) AS session_id
  FROM sessions
),

-- Aggregate session-level metrics
session_metrics AS (
  SELECT
    UserId,
    session_id,
    COUNTIF(
      LENGTH(message) >= 20
      OR message LIKE '%?%'
    ) AS real_queries,
    MAX(CASE WHEN Error = 'ERROR' THEN 1 ELSE 0 END) AS has_error
  FROM session_ids
  GROUP BY UserId, session_id
)

-- Final KPI calculation
SELECT
  ROUND(
    SAFE_DIVIDE(
      COUNTIF(real_queries BETWEEN 1 AND 2 AND has_error = 0),
      COUNT(*)
    ),
    4
  ) AS satisfactory_response_rate
FROM session_metrics;
```

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# KPI Data
df = pd.DataFrame({
    "KPI": ["Satisfactory Response Rate"],
    "Value": [0.39]
})

# Plot
plt.figure(figsize=(6, 2))
plt.barh(
    df["KPI"],
    df["Value"],
    color="#1f77b4"
)

plt.xlim(0, 1)
plt.title("Satisfactory Response Rate", fontsize=12)
plt.xlabel("Session Proportion")
plt.ylabel("")

# Value label
plt.text(
    df["Value"][0] + 0.02,
    0,
    "39%",
    va="center",
    fontsize=11
)

plt.tight_layout()
plt.show()

### KPI 2 — Reformulation Rate

**Value:** 70%

**Definition:**
Percentage of sessions where the user performs more than one real query.

A session is considered reformulated if it contains more than one real query (real queries are defined as messages with ≥20 characters or containing a question mark).

**Interpretation:**
Most users need to insist, refine, or reformulate their request before obtaining a satisfactory answer.  

This suggests friction in clarity, response structure, or answer prioritization rather than purely technical failure.

#### SQL Used to Calculate KPI 2

The following query was executed in BigQuery to calculate the Reformulation Rate:

```sql
WITH base AS (
  SELECT
    UserId,
    event_ts,
    message,
    LENGTH(message) AS msg_length,
    LAG(event_ts) OVER (
      PARTITION BY UserId
      ORDER BY event_ts
    ) AS prev_ts
  FROM `conductive-bank-469015-r4.esfera_IA.logs_ai_esfera_producto_vw`
),

-- Filter trivial messages:
-- Include only messages that:
--   - have 20 or more characters
--   - OR contain a question mark
filtered_base AS (
  SELECT *
  FROM base
  WHERE msg_length >= 20
     OR message LIKE '%?%'
),

-- Identify new sessions using a 10-minute inactivity threshold
sessions AS (
  SELECT
    UserId,
    event_ts,
    IF(
      prev_ts IS NULL
      OR TIMESTAMP_DIFF(event_ts, prev_ts, MINUTE) > 10,
      1,
      0
    ) AS new_session_flag
  FROM filtered_base
),

-- Assign cumulative session IDs
session_ids AS (
  SELECT
    UserId,
    event_ts,
    SUM(new_session_flag) OVER (
      PARTITION BY UserId
      ORDER BY event_ts
    ) AS session_id
  FROM sessions
),

-- Count number of real queries per session
session_counts AS (
  SELECT
    UserId,
    session_id,
    COUNT(*) AS queries_in_session
  FROM session_ids
  GROUP BY UserId, session_id
)

-- Final KPI calculation:
-- Sessions with more than one real query divided by total sessions
SELECT
  COUNTIF(queries_in_session > 1) / COUNT(*) AS reformulation_rate
FROM session_counts;
```

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# KPI Data
df = pd.DataFrame({
    "KPI": ["Reformulated Sessions"],
    "Value": [0.70]
})

# Plot
plt.figure(figsize=(6, 2))
plt.barh(
    df["KPI"],
    df["Value"],
    color="#ff7f0e"  # subtle orange to differentiate from KPI 1
)

plt.xlim(0, 1)
plt.title("Reformulation Rate", fontsize=12)
plt.xlabel("Session Proportion")
plt.ylabel("")

# Value label
plt.text(
    df["Value"][0] + 0.02,
    0,
    "70%",
    va="center",
    fontsize=11
)

plt.tight_layout()
plt.show()

### KPI 3 — Average Queries per Session

**Value**: 5.36 queries

**Definition:**
Average number of real queries per session.

**Interpretation:**
Conversations tend to involve multiple interactions, suggesting users refine, expand, or deepen their requests within the same session.

#### SQL Used to Calculate KPI 3

The following query was executed in BigQuery to calculate the average number of real queries per session:

```sql
    WITH base AS (
  SELECT
    UserId,
    event_ts,
    message,
    LENGTH(message) AS msg_length,
    LAG(event_ts) OVER (
      PARTITION BY UserId
      ORDER BY event_ts
    ) AS prev_ts
  FROM `conductive-bank-469015-r4.esfera_IA.logs_ai_esfera_producto_vw`
),

-- Keep only real queries
filtered_base AS (
  SELECT *
  FROM base
  WHERE msg_length >= 20
     OR message LIKE '%?%'
),

-- Identify session boundaries (10-minute rule)
sessions AS (
  SELECT
    UserId,
    event_ts,
    IF(
      prev_ts IS NULL
      OR TIMESTAMP_DIFF(event_ts, prev_ts, MINUTE) > 10,
      1,
      0
    ) AS new_session_flag
  FROM filtered_base
),

-- Assign session IDs
session_ids AS (
  SELECT
    UserId,
    event_ts,
    SUM(new_session_flag) OVER (
      PARTITION BY UserId
      ORDER BY event_ts
    ) AS session_id
  FROM sessions
),

-- Count queries per session
session_counts AS (
  SELECT
    UserId,
    session_id,
    COUNT(*) AS queries_in_session
  FROM session_ids
  GROUP BY UserId, session_id
)

SELECT
  AVG(queries_in_session) AS avg_queries_per_session
FROM session_counts;
    ```

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# KPI data
df = pd.DataFrame({
    "KPI": ["Average Queries per Session"],
    "Value": [5.36]
})

# Plot
plt.figure(figsize=(4, 4))
plt.bar(
    df["KPI"],
    df["Value"],
    color="#7f7f7f"
)

plt.title("Average Queries per Session", fontsize=12)
plt.ylabel("Number of Queries")
plt.ylim(0, 6)

# Value label
plt.text(
    0,
    df["Value"][0] + 0.15,
    "5.36",
    ha="center",
    fontsize=11
)

plt.tight_layout()
plt.show()

### KPI 4 — Explicit Error Rate

**Value**: 0.9%

**Definition**:
Percentage of interactions where at least one explicit technical error was recorded by the system (Error = 'ERROR').

**Interpretation**:
The explicit technical error rate is low (~0.9%), indicating system stability during the analyzed period. This metric serves as a technical reliability baseline for future monitoring.

#### SQL Used to Calculate KPI 4

The following query was executed in BigQuery to calculate the Explicit Error Rate:

```sql
SELECT
  COUNTIF(TRIM(UPPER(Error)) = 'ERROR') / COUNT(*) AS explicit_error_rate
FROM `conductive-bank-469015-r4.esfera_IA.logs_ai_esfera_producto_vw`;
```

In [ ]:
import matplotlib.pyplot as plt

# KPI value (replace if recalculated)
error_rate = 0.009  # 0.9%

plt.figure(figsize=(4, 2))

plt.text(
    0.5, 0.6,
    f"{error_rate*100:.1f}%",
    ha="center",
    va="center",
    fontsize=32,
    color="#d62728"  # soft red for errors
)

plt.text(
    0.5, 0.25,
    "Explicit Error Rate",
    ha="center",
    va="center",
    fontsize=12
)

plt.axis("off")
plt.title("KPI 4 — Explicit Error Rate", fontsize=12)
plt.show()

### KPI 5 — AI Response Time Performance

**Value:**

- Average TTFT: ~0.77 seconds
- Average Total Duration: ~6.96 seconds
    
(Initial sample — exploratory baseline)


**Definition:**

This KPI measures how long the AI takes:
- To start responding (TTFT – Time To First Token)
- To complete the full response (Total Duration)

Metrics are calculated in milliseconds and converted to seconds for interpretation.

**Interpretation:**

- The AI starts responding in under 1 second, which supports a strong perception of responsiveness.
- The total response duration is higher, which is expected for analytical and multi-step answers.
- This KPI establishes a performance baseline for future monitoring as usage scales.

⸻

***Methodological Note:***

Values represent an initial usage sample and should be interpreted as a performance baseline, not as a statistically conclusive benchmark.

#### SQL Used to Calculate KPI 5

The following query was executed in BigQuery to calculate the AI Response Time Performance:

```sql
SELECT
  COUNT(*) AS total_queries,
  ROUND(AVG(ttft), 2) AS avg_ttft_ms,
  ROUND(AVG(duration_ms), 2) AS avg_duration_ms,
  ROUND(APPROX_QUANTILES(ttft, 100)[OFFSET(50)], 2) AS median_ttft_ms,
  ROUND(APPROX_QUANTILES(duration_ms, 100)[OFFSET(50)], 2) AS median_duration_ms
FROM `conductive-bank-469015-r4.esfera_IA.logs_ai_esfera_tiempos_tokens_vw`;
```

In [ ]:
import matplotlib.pyplot as plt

# Baseline values converted to seconds
labels = ['TTFT (Response Start)', 'Total Duration']
values = [0.77, 6.96]  # seconds

plt.figure(figsize=(5, 3))
bars = plt.bar(labels, values, color=['#1f77b4', '#ff7f0e'])

plt.ylabel('Seconds')
plt.title('Average AI Response Time')

# Add value labels above bars
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        height,
        f'{height:.2f}s',
        ha='center',
        va='bottom'
    )

plt.ylim(0, max(values) * 1.3)
plt.tight_layout()
plt.show()

### KPI 6 — Cost per Query

**Value:** 
USD 0.1924 per query

**Definition:**

Average estimated cost in USD per AI query, calculated based on input tokens (prompt) and output tokens (response), using the pricing model applied at the time of analysis.

**Interpretation:**

The average cost per query reflects relatively long and analytical responses.
At this stage, the metric serves as a financial baseline to monitor efficiency, scalability, and optimization opportunities as usage grows.

⸻

***Methodological Note:***

Values represent an initial usage sample and should be interpreted as a cost baseline, not as a scaled production benchmark.



#### SQL Used to Calculate KPI 6

The following query was executed in BigQuery to estimate the average and total cost based on token consumption:

```sql
SELECT
  COUNT(*) AS total_consultas,
  ROUND(
    AVG(
      (prompt_token_count / 1000000.0 * 0.50) +
      (candidates_token_count / 1000000.0 * 3.00)
    ),
    6
  ) AS avg_cost_per_query_usd,
  ROUND(
    SUM(
      (prompt_token_count / 1000000.0 * 0.50) +
      (candidates_token_count / 1000000.0 * 3.00)
    ),
    6
  ) AS total_cost_usd
FROM `conductive-bank-469015-r4.esfera_IA.logs_ai_esfera_tiempos_tokens_vw`;
```

In [ ]:
import matplotlib.pyplot as plt

# KPI value 
avg_cost = 0.192402  # USD per query

plt.figure(figsize=(4, 2))

plt.text(
    0.5, 0.6,
    f"${avg_cost:.4f}",
    ha="center",
    va="center",
    fontsize=32,
    color="#9467bd"
)

plt.text(
    0.5, 0.25,
    "Average cost per query",
    ha="center",
    va="center",
    fontsize=12
)

plt.axis("off")
plt.title("KPI 6 — Cost per Query (USD)", fontsize=12)

plt.show()

## 8. Strategic Insights

Based on the exploratory analysis and KPI results, several strategic conclusions emerge:

1.	**The AI is primarily used as an analytical assistant**, not as an operational tool.
Most queries relate to financial, project progress, and reporting topics.
2. **Technical stability is strong (low explicit error rate)**, but satisfaction remains moderate.
This suggests that friction is more related to response clarity and synthesis than system reliability.
3. **High reformulation rate (70%) indicates iteration behavior**, meaning users often refine or clarify their requests.
4. **Average queries per session (5.36) reflect complex interactions**, not one-shot questions.
5. **Cost and response time establish an operational baseline** for future monitoring and scaling decisions.

## 9. Monitoring & Dashboard Integration

The defined KPIs were operationalized into interactive dashboards built in Tableau.

Two complementary dashboard layers were designed:

### Dashboard 1 – System Status & Usage
Provides a consolidated executive view including:
- Usage distribution
- Satisfactory response rate
- Reformulation rate
- Explicit error rate
- Cost per query

Designed for product, leadership, and operational monitoring.

### Dashboard 2 – Technical Performance Exploration
Focused on:
- TTFT vs total response duration
- Temporal evolution structure
- Baseline performance tracking

This layer supports longitudinal monitoring and performance scalability as usage grows.

Together, these dashboards translate analytical definitions into continuous monitoring infrastructure.

## 10. Conclusion

This project transforms raw conversational system logs into a structured analytical framework capable of measuring:

- Usage patterns
- Interaction quality
- Technical stability
- Operational cost

The system is technically stable and actively used for analytical purposes. 
Current improvement opportunities are concentrated in response clarity, synthesis quality, and user guidance rather than infrastructure reliability.

The defined KPIs and monitoring structure provide a scalable foundation for continuous evaluation as adoption increases.